In [14]:
import sys
from importlib import reload
import logging
import csv
from nltk.stem.porter import PorterStemmer
import string
import gensim
import pickle
from gensim.summarization.bm25 import BM25


porterStemmer = PorterStemmer()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def removeNonAscii(s):
    return "".join(filter(lambda x:ord(x) < 128, s))


def preprocess(passage):
    clean_passage = passage.lower()
    clean_passage = removeNonAscii(clean_passage)
    clean_passage = clean_passage.translate(str.maketrans('', '', string.punctuation)) 
    
    clean_passage_list = clean_passage.split()
    clean_passage_list_stem = []
    for word in clean_passage_list:
        if word not in gensim.parsing.preprocessing.STOPWORDS:
            clean_passage_list_stem.append(porterStemmer.stem(word))
    return clean_passage_list_stem


C:\Utilities\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [15]:
from collections import defaultdict
import csv


In [2]:
collections = defaultdict(str)
with open("C:\\Users\\sundaras\\AnacondaProjects\\OneML\\data\\collection_cleaned.tsv") as fr:
    reader = csv.reader(fr, delimiter="\t")
    for row in reader:
        collections[int(row[0])] = row[1]

In [3]:
from collections import defaultdict
import csv

query_text = defaultdict(str)
with open("C:\\Users\\sundaras\\AnacondaProjects\\OneML\\data\\queries.dev.tsv", encoding="UTF8") as fr:
    reader = csv.reader(fr, delimiter="\t")
    for row in reader:
        query_text[int(row[0])] = row[1]

In [4]:
qrels = defaultdict(list)
with open("C:\\Users\\sundaras\\AnacondaProjects\\OneML\\data\\qrels.dev.tsv") as fr:
    reader = csv.reader(fr, delimiter="\t")
    for row in reader:
        qrels[int(row[0])].append(int(row[2]))

qrels_nr = defaultdict(list)
with open("C:\\Users\\sundaras\\AnacondaProjects\\OneML\\data\\qrels_nr.dev.tsv") as fr:
    reader = csv.reader(fr, delimiter="\t")
    for row in reader:
        qrels_nr[int(row[0])].append(int(row[2]))

In [12]:
import pickle

with open("C:\\Users\\mapyredd\\Documents\\marco\\data\\model\\bm25.pkl", 'rb') as fr:
    bm25 = pickle.load(fr)
    
average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys()) 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\mapyredd\\Documents\\marco\\data\\model\\bm25.pkl'

In [6]:
def getBM25Feature():
    # key: (qid, docid, rel)
    # value: score 
    scores = defaultdict(float)
    for qid, docid_list in qrels.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)
        for docid in docid_list:
            score = bm25.get_score(query_list, docid, average_idf)
            scores[(qid, docid, 1)] = score
            
    for qid, docid_list in qrels_nr.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)
        for docid in docid_list:
            score = bm25.get_score(query_list, docid, average_idf)
            scores[(qid, docid, 0)] = score
    return scores

In [7]:
def getDocLengthFeature():
    # key: (qid, docid, rel)
    # value: doc length
    scores = defaultdict(float)
    for qid, docid_list in qrels.items():
        for docid in docid_list:
            score = len(collections[docid].split())
            scores[(qid, docid, 1)] = score
            
    for qid, docid_list in qrels_nr.items():
        for docid in docid_list:
            score = len(collections[docid].split())
            scores[(qid, docid, 0)] = score
    return scores

In [8]:
def getCoverageFeature():
    # key: (qid, docid, rel)
    # value: score -> contains query term coverage and ratio
    zero_length_queries = 0
    scores = defaultdict(tuple)
    for qid, docid_list in qrels.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)
        for docid in docid_list:
            score = len(set(query_list).intersection(set(collections[docid].split())))
            scores[(qid, docid, 1)] = (score, float(score)/(len(query_list) + 1))
            if len(query_list) == 0:
                zero_length_queries += 1
    
    print (zero_length_queries)
    
    for qid, docid_list in qrels_nr.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)
        for docid in docid_list:
            score = len(set(query_list).intersection(set(collections[docid].split())))
            scores[(qid, docid, 0)] = (score, float(score)/(len(query_list) + 1))
            
    return scores

In [5]:
def getAndFeature():
    # key: (qid, docid, rel)
    # value: score -> contains query term converage and ratio
    scores = defaultdict(float)
    for qid, docid_list in qrels.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)
        for docid in docid_list:
            terms = collections[docid].split()
            andScore = set(query_list).issubset(terms)
            scores[(qid, docid, 1)] = 1 if andScore else 0
            
    for qid, docid_list in qrels_nr.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)
        for docid in docid_list:
            terms = collections[docid].split()
            andScore = set(query_list).issubset(terms)
            scores[(qid, docid, 0)] = 1 if andScore else 0
            
    return scores      
            

In [10]:
def getAnd2Feature():
    # key: (qid, docid, rel)
    # value: score -> contains query term converage and ratio
    scores = defaultdict(float)
    for qid, docid_list in qrels.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)
        for docid in docid_list:
            terms = collections[docid].split()
            total_word_count = 0
            all_verbs_found = True
            for query in query_list:
                word_count = sum(term == query for term in terms)
                if word_count == 0:
                    all_verbs_found = False
                    break
                else:
                    total_word_count = total_word_count + word_count
                    
            scores[(qid, docid, 1)] = total_word_count if all_verbs_found else 0
                
    for qid, docid_list in qrels_nr.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)
        for docid in docid_list:
            terms = collections[docid].split()
            total_word_count = 0
            all_verbs_found = True
            for query in query_list:
                word_count = sum(term == query for term in terms)
                if word_count == 0:
                    all_verbs_found = False
                    break
                else:
                    total_word_count = total_word_count + word_count
                    
            scores[(qid, docid, 0)] = total_word_count if all_verbs_found else 0 
        
    return scores

In [30]:
import sys
import csv
import logging
import pandas as pd
import gensim
from gensim import corpora

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
csv.field_size_limit(10000000)


# Load raw text corpus and yield preprocessed text
class MyTextCorpus(object):
    def __init__(self, inputFile):
        self.inputFile = inputFile
        
    def __iter__(self):
        with open(self.inputFile, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter='\t')
            for row in reader:
                yield row[1].split()


inputFile = "C:\\Users\\mapyredd\\Documents\\marco\\data\\collection_cleaned.tsv"
outputFile = "C:\\Users\\mapyredd\\Documents\\marco\\data\\collection_cleaned.dict"
corpus_text_memory_friendly = MyTextCorpus(inputFile)

# remove stop words and numbers. Form a dictionary
dictionary = corpora.Dictionary([doc for doc in corpus_text_memory_friendly])
dictionary.save(outputFile)

2018-07-21 21:44:03,664 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-07-21 21:44:05,293 : INFO : adding document #10000 to Dictionary(26896 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:44:06,217 : INFO : adding document #20000 to Dictionary(41470 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:44:07,746 : INFO : adding document #30000 to Dictionary(54299 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:44:09,040 : INFO : adding document #40000 to Dictionary(65605 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:44:10,363 : INFO : adding document #50000 to Dictionary(76310 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:44:11,462 : INFO : adding document #60000 to Dictionary(86163 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:44:13,759 : INFO : adding document #70000 to Dictiona

2018-07-21 21:44:55,658 : INFO : adding document #580000 to Dictionary(450015 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:44:56,499 : INFO : adding document #590000 to Dictionary(455943 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:44:57,846 : INFO : adding document #600000 to Dictionary(461800 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:44:58,882 : INFO : adding document #610000 to Dictionary(467317 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:44:59,847 : INFO : adding document #620000 to Dictionary(472992 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:45:02,889 : INFO : adding document #630000 to Dictionary(478747 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:45:04,409 : INFO : adding document #640000 to Dictionary(484486 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)

2018-07-21 21:45:54,554 : INFO : adding document #1160000 to Dictionary(757372 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:45:55,349 : INFO : adding document #1170000 to Dictionary(762525 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:45:55,969 : INFO : adding document #1180000 to Dictionary(767639 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:45:56,478 : INFO : adding document #1190000 to Dictionary(772451 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:45:56,988 : INFO : adding document #1200000 to Dictionary(777549 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:45:59,632 : INFO : adding document #1210000 to Dictionary(782563 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:46:00,176 : INFO : adding document #1220000 to Dictionary(787305 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commu

2018-07-21 21:46:44,751 : INFO : adding document #1730000 to Dictionary(1023766 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:46:45,861 : INFO : adding document #1740000 to Dictionary(1028258 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:46:46,980 : INFO : adding document #1750000 to Dictionary(1032339 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:46:48,000 : INFO : adding document #1760000 to Dictionary(1036972 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:46:49,139 : INFO : adding document #1770000 to Dictionary(1041506 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:46:50,203 : INFO : adding document #1780000 to Dictionary(1045803 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:46:51,954 : INFO : adding document #1790000 to Dictionary(1050149 unique tokens: ['achiev', 'amid', 'atom', 'cloud',

2018-07-21 21:47:40,886 : INFO : adding document #2300000 to Dictionary(1265489 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:47:41,562 : INFO : adding document #2310000 to Dictionary(1269245 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:47:42,187 : INFO : adding document #2320000 to Dictionary(1273429 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:47:42,926 : INFO : adding document #2330000 to Dictionary(1277516 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:47:43,607 : INFO : adding document #2340000 to Dictionary(1281615 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:47:44,506 : INFO : adding document #2350000 to Dictionary(1285647 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 21:47:45,184 : INFO : adding document #2360000 to Dictionary(1290049 unique tokens: ['achiev', 'amid', 'atom', 'cloud',

...

2018-07-21 22:22:03,449 : INFO : adding document #8280000 to Dictionary(2000000 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 22:22:05,631 : INFO : discarding 3286 tokens: [('degalz', 1), ('6515015217976', 1), ('restricct', 1), ('p1d', 1), ('2050the', 1), ('10112669', 1), ('4222574475', 1), ('3411b1', 1), ('httpwwwdolgovdoltopicbenefitsleavefmlahtm', 1), ('minutesladl', 1)]...
2018-07-21 22:22:05,632 : INFO : keeping 2000000 tokens which were in no less than 0 and no more than 8290000 (=100.0%) documents
2018-07-21 22:22:07,992 : INFO : resulting dictionary: Dictionary(2000000 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 22:22:08,027 : INFO : adding document #8290000 to Dictionary(2000000 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 22:22:10,171 : INFO : discarding 3268 tokens: [('refinin', 1), ('jrfree', 1), ('wwwgalileebiblecamporg', 1), ('christfocus', 1), ('aaaah', 1), ('itemalbummount', 1),

2018-07-21 22:23:02,583 : INFO : discarding 3448 tokens: [('makemytripcomswitzerland', 1), ('davosklost', 1), ('4819190', 1), ('boltic', 1), ('sinivi', 1), ('oyema', 1), ('iochav', 1), ('httpwwwfuntriviacomaskftquestion31647htmlth', 1), ('httpblogstechnetcombnetworkingarchive20081209dhcpshowsthependingupdateiconevenafterthearecordisaddedtodnsaspx', 1), ('disrip', 1)]...
2018-07-21 22:23:02,584 : INFO : keeping 2000000 tokens which were in no less than 0 and no more than 8410000 (=100.0%) documents
2018-07-21 22:23:04,984 : INFO : resulting dictionary: Dictionary(2000000 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 22:23:05,030 : INFO : adding document #8410000 to Dictionary(2000000 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 22:23:07,215 : INFO : discarding 3340 tokens: [('paintcheck', 1), ('aspectswhen', 1), ('complicatedsom', 1), ('designhen', 1), ('ideasak', 1), ('professionalsnow', 1), ('budgetefurbish', 1), ('yourselfl

2018-07-21 22:23:58,420 : INFO : discarding 3305 tokens: [('diphoshph', 1), ('lubusz', 1), ('pietruszkaepa', 1), ('xhtmlhtml', 1), ('cuerpocuatro', 1), ('lengthhalf', 1), ('recorri', 1), ('legislationcirt', 1), ('memberstrust', 1), ('processaragraph', 1)]...
2018-07-21 22:23:58,421 : INFO : keeping 2000000 tokens which were in no less than 0 and no more than 8530000 (=100.0%) documents
2018-07-21 22:24:02,110 : INFO : resulting dictionary: Dictionary(2000000 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 22:24:02,173 : INFO : adding document #8530000 to Dictionary(2000000 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 22:24:05,417 : INFO : discarding 3068 tokens: [('mamaaugust', 1), ('httpwwwacronymfindercommilitaryandgovernmentscarhtml', 1), ('producepack', 1), ('aubarachnoid', 1), ('035040', 1), ('mlmin1', 1), ('systemsubarachnoid', 1), ('subaracchnoid', 1), ('10k12k14k18k', 1), ('7121988', 1)]...
2018-07-21 22:24:05,418 : INF

2018-07-21 22:24:55,915 : INFO : discarding 3312 tokens: [('electrodetrictli', 1), ('perpetratora', 1), ('2a2middl', 1), ('warshipul', 1), ('opticsreadi', 1), ('anticoagulantsacetaminophen', 1), ('threelength', 1), ('elucidateverb', 1), ('jnan', 1), ('nosqlfrom', 1)]...
2018-07-21 22:24:55,915 : INFO : keeping 2000000 tokens which were in no less than 0 and no more than 8650000 (=100.0%) documents
2018-07-21 22:24:58,295 : INFO : resulting dictionary: Dictionary(2000000 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 22:24:58,331 : INFO : adding document #8650000 to Dictionary(2000000 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 22:25:00,779 : INFO : discarding 3223 tokens: [('uscist', 1), ('visasoth', 1), ('abroadalidityexpir', 1), ('validity2', 1), ('14413906', 1), ('clearbuy', 1), ('daysblast', 1), ('protectionarg', 1), ('shopvacuum', 1), ('cleanlinesso', 1)]...
2018-07-21 22:25:00,780 : INFO : keeping 2000000 tokens which w

2018-07-21 22:25:56,398 : INFO : keeping 2000000 tokens which were in no less than 0 and no more than 8770000 (=100.0%) documents
2018-07-21 22:25:58,789 : INFO : resulting dictionary: Dictionary(2000000 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 22:25:58,824 : INFO : adding document #8770000 to Dictionary(2000000 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 22:26:01,525 : INFO : discarding 3039 tokens: [('fuyuanshui', 1), ('guoyou', 1), ('huadu', 1), ('huoxiu', 1), ('flippedlick', 1), ('autorotation6hen', 1), ('orientationnstruct', 1), ('backhelp', 1), ('questionheck', 1), ('automaticallyheck', 1)]...
2018-07-21 22:26:01,526 : INFO : keeping 2000000 tokens which were in no less than 0 and no more than 8780000 (=100.0%) documents
2018-07-21 22:26:04,396 : INFO : resulting dictionary: Dictionary(2000000 unique tokens: ['achiev', 'amid', 'atom', 'cloud', 'commun']...)
2018-07-21 22:26:04,435 : INFO : adding document #8780000

In [9]:
from gensim.models import TfidfModel
from gensim import corpora
from gensim.corpora import Dictionary

dictionary = corpora.Dictionary.load("C:\\Users\\mapyredd\\Documents\\marco\\data\\collection_cleaned.dict")

class MyCorpus(object):
    def __iter__(self):
        with open('C:\\Users\\mapyredd\\Documents\\marco\\data\\collection_cleaned.tsv') as fr:
            reader = csv.reader(fr, delimiter = "\t")
            for row in reader:
                # assume there's one document per line, tokens separated by whitespace
                yield dictionary.doc2bow(row[1].lower().split())

tfidfmodel = TfidfModel(MyCorpus())

2018-07-22 15:54:56,511 : INFO : loading Dictionary object from C:\Users\mapyredd\Documents\marco\data\collection_cleaned.dict
2018-07-22 15:54:59,314 : INFO : loaded C:\Users\mapyredd\Documents\marco\data\collection_cleaned.dict
2018-07-22 15:54:59,329 : INFO : collecting document frequencies
2018-07-22 15:54:59,341 : INFO : PROGRESS: processing document #0
2018-07-22 15:55:00,370 : INFO : PROGRESS: processing document #10000
2018-07-22 15:55:01,817 : INFO : PROGRESS: processing document #20000
2018-07-22 15:55:02,965 : INFO : PROGRESS: processing document #30000
2018-07-22 15:55:04,041 : INFO : PROGRESS: processing document #40000
2018-07-22 15:55:05,070 : INFO : PROGRESS: processing document #50000
2018-07-22 15:55:05,988 : INFO : PROGRESS: processing document #60000
2018-07-22 15:55:07,246 : INFO : PROGRESS: processing document #70000
2018-07-22 15:55:08,517 : INFO : PROGRESS: processing document #80000
2018-07-22 15:55:09,143 : INFO : PROGRESS: processing document #90000
2018-07-2

2018-07-22 15:57:09,147 : INFO : PROGRESS: processing document #1120000
2018-07-22 15:57:09,531 : INFO : PROGRESS: processing document #1130000
2018-07-22 15:57:09,931 : INFO : PROGRESS: processing document #1140000
2018-07-22 15:57:10,349 : INFO : PROGRESS: processing document #1150000
2018-07-22 15:57:11,081 : INFO : PROGRESS: processing document #1160000
2018-07-22 15:57:11,464 : INFO : PROGRESS: processing document #1170000
2018-07-22 15:57:12,056 : INFO : PROGRESS: processing document #1180000
2018-07-22 15:57:13,271 : INFO : PROGRESS: processing document #1190000
2018-07-22 15:57:15,857 : INFO : PROGRESS: processing document #1200000
2018-07-22 15:57:16,284 : INFO : PROGRESS: processing document #1210000
2018-07-22 15:57:16,665 : INFO : PROGRESS: processing document #1220000
2018-07-22 15:57:17,866 : INFO : PROGRESS: processing document #1230000
2018-07-22 15:57:18,737 : INFO : PROGRESS: processing document #1240000
2018-07-22 15:57:19,569 : INFO : PROGRESS: processing document #

2018-07-22 15:58:49,359 : INFO : PROGRESS: processing document #2260000
2018-07-22 15:58:49,696 : INFO : PROGRESS: processing document #2270000
2018-07-22 15:58:50,054 : INFO : PROGRESS: processing document #2280000
2018-07-22 15:58:50,419 : INFO : PROGRESS: processing document #2290000
2018-07-22 15:58:50,853 : INFO : PROGRESS: processing document #2300000
2018-07-22 15:58:51,214 : INFO : PROGRESS: processing document #2310000
2018-07-22 15:58:51,575 : INFO : PROGRESS: processing document #2320000
2018-07-22 15:58:51,966 : INFO : PROGRESS: processing document #2330000
2018-07-22 15:58:52,374 : INFO : PROGRESS: processing document #2340000
2018-07-22 15:58:52,783 : INFO : PROGRESS: processing document #2350000
2018-07-22 15:58:53,236 : INFO : PROGRESS: processing document #2360000
2018-07-22 15:58:53,610 : INFO : PROGRESS: processing document #2370000
2018-07-22 15:58:54,014 : INFO : PROGRESS: processing document #2380000
2018-07-22 15:58:54,556 : INFO : PROGRESS: processing document #

2018-07-22 15:59:37,061 : INFO : PROGRESS: processing document #3400000
2018-07-22 15:59:37,444 : INFO : PROGRESS: processing document #3410000
2018-07-22 15:59:37,921 : INFO : PROGRESS: processing document #3420000
2018-07-22 15:59:38,420 : INFO : PROGRESS: processing document #3430000
2018-07-22 15:59:38,793 : INFO : PROGRESS: processing document #3440000
2018-07-22 15:59:39,155 : INFO : PROGRESS: processing document #3450000
2018-07-22 15:59:39,538 : INFO : PROGRESS: processing document #3460000
2018-07-22 15:59:39,908 : INFO : PROGRESS: processing document #3470000
2018-07-22 15:59:40,328 : INFO : PROGRESS: processing document #3480000
2018-07-22 15:59:40,707 : INFO : PROGRESS: processing document #3490000
2018-07-22 15:59:41,076 : INFO : PROGRESS: processing document #3500000
2018-07-22 15:59:41,505 : INFO : PROGRESS: processing document #3510000
2018-07-22 15:59:41,873 : INFO : PROGRESS: processing document #3520000
2018-07-22 15:59:42,241 : INFO : PROGRESS: processing document #

2018-07-22 16:00:28,727 : INFO : PROGRESS: processing document #4540000
2018-07-22 16:00:29,155 : INFO : PROGRESS: processing document #4550000
2018-07-22 16:00:29,553 : INFO : PROGRESS: processing document #4560000
2018-07-22 16:00:29,909 : INFO : PROGRESS: processing document #4570000
2018-07-22 16:00:30,322 : INFO : PROGRESS: processing document #4580000
2018-07-22 16:00:30,751 : INFO : PROGRESS: processing document #4590000
2018-07-22 16:00:31,125 : INFO : PROGRESS: processing document #4600000
2018-07-22 16:00:31,538 : INFO : PROGRESS: processing document #4610000
2018-07-22 16:00:31,951 : INFO : PROGRESS: processing document #4620000
2018-07-22 16:00:32,346 : INFO : PROGRESS: processing document #4630000
2018-07-22 16:00:32,706 : INFO : PROGRESS: processing document #4640000
2018-07-22 16:00:33,122 : INFO : PROGRESS: processing document #4650000
2018-07-22 16:00:33,510 : INFO : PROGRESS: processing document #4660000
2018-07-22 16:00:33,906 : INFO : PROGRESS: processing document #

2018-07-22 16:01:23,537 : INFO : PROGRESS: processing document #5680000
2018-07-22 16:01:23,922 : INFO : PROGRESS: processing document #5690000
2018-07-22 16:01:24,336 : INFO : PROGRESS: processing document #5700000
2018-07-22 16:01:24,815 : INFO : PROGRESS: processing document #5710000
2018-07-22 16:01:25,194 : INFO : PROGRESS: processing document #5720000
2018-07-22 16:01:25,583 : INFO : PROGRESS: processing document #5730000
2018-07-22 16:01:25,955 : INFO : PROGRESS: processing document #5740000
2018-07-22 16:01:26,334 : INFO : PROGRESS: processing document #5750000
2018-07-22 16:01:26,736 : INFO : PROGRESS: processing document #5760000
2018-07-22 16:01:27,211 : INFO : PROGRESS: processing document #5770000
2018-07-22 16:01:27,620 : INFO : PROGRESS: processing document #5780000
2018-07-22 16:01:28,050 : INFO : PROGRESS: processing document #5790000
2018-07-22 16:01:28,485 : INFO : PROGRESS: processing document #5800000
2018-07-22 16:01:28,867 : INFO : PROGRESS: processing document #

2018-07-22 16:02:22,935 : INFO : PROGRESS: processing document #6820000
2018-07-22 16:02:23,631 : INFO : PROGRESS: processing document #6830000
2018-07-22 16:02:24,336 : INFO : PROGRESS: processing document #6840000
2018-07-22 16:02:25,048 : INFO : PROGRESS: processing document #6850000
2018-07-22 16:02:25,771 : INFO : PROGRESS: processing document #6860000
2018-07-22 16:02:26,428 : INFO : PROGRESS: processing document #6870000
2018-07-22 16:02:27,194 : INFO : PROGRESS: processing document #6880000
2018-07-22 16:02:27,737 : INFO : PROGRESS: processing document #6890000
2018-07-22 16:02:28,242 : INFO : PROGRESS: processing document #6900000
2018-07-22 16:02:28,796 : INFO : PROGRESS: processing document #6910000
2018-07-22 16:02:29,303 : INFO : PROGRESS: processing document #6920000
2018-07-22 16:02:29,784 : INFO : PROGRESS: processing document #6930000
2018-07-22 16:02:30,257 : INFO : PROGRESS: processing document #6940000
2018-07-22 16:02:30,733 : INFO : PROGRESS: processing document #

2018-07-22 16:03:17,635 : INFO : PROGRESS: processing document #7960000
2018-07-22 16:03:17,975 : INFO : PROGRESS: processing document #7970000
2018-07-22 16:03:18,340 : INFO : PROGRESS: processing document #7980000
2018-07-22 16:03:18,692 : INFO : PROGRESS: processing document #7990000
2018-07-22 16:03:19,090 : INFO : PROGRESS: processing document #8000000
2018-07-22 16:03:19,448 : INFO : PROGRESS: processing document #8010000
2018-07-22 16:03:19,807 : INFO : PROGRESS: processing document #8020000
2018-07-22 16:03:20,146 : INFO : PROGRESS: processing document #8030000
2018-07-22 16:03:20,496 : INFO : PROGRESS: processing document #8040000
2018-07-22 16:03:20,841 : INFO : PROGRESS: processing document #8050000
2018-07-22 16:03:21,179 : INFO : PROGRESS: processing document #8060000
2018-07-22 16:03:21,527 : INFO : PROGRESS: processing document #8070000
2018-07-22 16:03:21,862 : INFO : PROGRESS: processing document #8080000
2018-07-22 16:03:22,197 : INFO : PROGRESS: processing document #

In [10]:
def getIdfFeature():
    # key: (qid, docid, rel)
    # value: idf -> contains sum, max, min
    
    scores = defaultdict(tuple)
    for qid, docid_list in qrels.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)

        for docid in docid_list:
            score_list = []
            if len(query_list) == 0:
                score_list.append(0)
            for query_term in query_list:
                doctext = collections[docid].split()
                if query_term in doctext:
                    if dictionary.token2id.get(query_term) != None:
                        term_id = dictionary.token2id[query_term]
                        score_list.append(tfidfmodel.idfs[term_id])
                    else:
                        score_list.append(0)
                else:
                    score_list.append(0)
                    
            sum_idf = sum(score_list)
            max_idf = max(score_list)
            min_idf = min(score_list)

            scores[(qid, docid, 1)] = (sum_idf, max_idf, min_idf)
            
    for qid, docid_list in qrels_nr.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)

        for docid in docid_list:
            score_list = []
            if len(query_list) == 0:
                score_list.append(0)
            for query_term in query_list:
                doctext = collections[docid].split()
                if query_term in doctext:
                    if dictionary.token2id.get(query_term) != None:
                        term_id = dictionary.token2id[query_term]
                        score_list.append(tfidfmodel.idfs[term_id])
                    else:
                        score_list.append(0)
                else:
                    score_list.append(0)
                    
            sum_idf = sum(score_list)
            max_idf = max(score_list)
            min_idf = min(score_list)

            scores[(qid, docid, 0)] = (sum_idf, max_idf, min_idf)
            
    return scores

In [11]:
def getTfFeature():
    # key: (qid, docid, rel)
    # value: tf -> contains sum, max, min, mean
    
    scores = defaultdict(tuple)
    for qid, docid_list in qrels.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)          
        for docid in docid_list:
            score_list = []
            if len(query_list) == 0:
                score_list.append(0)
            for query_term in query_list:
                doctext = collections[docid].split()
                if query_term in doctext:
                    score_list.append(doctext.count(query_term))
                else:
                    score_list.append(0)

            sum_tf = sum(score_list)
            max_tf = max(score_list)
            min_tf = min(score_list)
            scores[(qid, docid, 1)] = (sum_tf, max_tf, min_tf)
            
    for qid, docid_list in qrels_nr.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)          
        for docid in docid_list:
            score_list = []
            if len(query_list) == 0:
                score_list.append(0)
            for query_term in query_list:
                doctext = collections[docid].split()
                if query_term in doctext:
                    score_list.append(doctext.count(query_term))
                else:
                    score_list.append(0)

            sum_tf = sum(score_list)
            max_tf = max(score_list)
            min_tf = min(score_list)
            scores[(qid, docid, 0)] = (sum_tf, max_tf, min_tf)
    return scores

In [12]:
def getTfIdfFeature():
    # key: (qid, docid, rel)
    # value: tf -> contains sum, max, min, mean
    scores = defaultdict(tuple)
    for qid, docid_list in qrels.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)
        for docid in docid_list:
            score_list = []
            if len(query_list) == 0:
                score_list.append(0.)              
            doctext = collections[docid].lower().split()
            doc_vector = tfidfmodel[dictionary.doc2bow(doctext)]            
            for query_term in query_list:
                if dictionary.token2id.get(query_term) != None:
                    s = [t[1] for t in doc_vector if t[0] == dictionary.token2id.get(query_term)]
                    if len(s) > 0:
                        score_list.append(s[0])
                    else:
                        score_list.append(0.)
                else:
                    score_list.append(0.)
            
            try:
                sum_tfidf = sum(score_list)
                max_tfidf = max(score_list)
                min_tfidf = min(score_list)
                mean_tfidf = sum_tfidf/len(doctext)
            except:
                import pdb;pdb.set_trace()
                
            scores[(qid, docid, 1)] = (sum_tfidf, max_tfidf, min_tfidf,mean_tfidf)

    
    for qid, docid_list in qrels_nr.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)
        for docid in docid_list:
            score_list = []
            if len(query_list) == 0:
                score_list.append(0)               
            doctext = collections[docid].lower().split()
            doc_vector = tfidfmodel[dictionary.doc2bow(doctext)]            
            for query_term in query_list:
                if dictionary.token2id.get(query_term) != None:
                    s = [t[1] for t in doc_vector if t[0] == dictionary.token2id.get(query_term)]
                    if len(s) > 0:
                        score_list.append(s[0])
                    else:
                        score_list.append(0.)
                else:
                    score_list.append(0.)
            
            sum_tfidf = sum(score_list)
            max_tfidf = max(score_list)
            min_tfidf = min(score_list)
            mean_tfidf = sum_tfidf/len(doctext)
                
            scores[(qid, docid, 0)] = (sum_tfidf, max_tfidf, min_tfidf,mean_tfidf)
    return scores

In [11]:
def getQueryLengthFeature():
    # key: (qid, docid, rel)
    # value: query length
    scores = defaultdict(tuple)
    for qid, docid_list in qrels.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)
        for docid in docid_list:
            scores[(qid, docid, 1)] = len(query_list)

    
    for qid, docid_list in qrels_nr.items():
        qtext = query_text[qid]
        query_list = preprocess(qtext)
        for docid in docid_list:
            scores[(qid, docid, 0)] = len(query_list)
    return scores

In [11]:
import pickle

# Combining all the features

# bm25_scores = getBM25Feature()
# pickle.dump( bm25_scores, open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\bm25_features_dev.p", "wb" ) )

# doclen_scores = getDocLengthFeature()
# pickle.dump( doclen_scores, open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\doclen_features_dev.p", "wb" ) )

# coverage_scores = getCoverageFeature()
# pickle.dump( coverage_scores, open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\coverage_features_dev.p", "wb" ) )

# idf_scores = getIdfFeature()
# pickle.dump( idf_scores, open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\idf_features_dev.p", "wb" ) )

# tf_scores = getTfFeature()
# pickle.dump( tf_scores, open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\tf_features_dev.p", "wb" ) )

# tfidf_scores = getTfIdfFeature()
# pickle.dump( tfidf_scores, open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\tfidf_features_dev.p", "wb" ) )

# querylen_scores = getQueryLengthFeature()
# pickle.dump( querylen_scores, open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\querylen_features_dev.p", "wb" ))

and_scores = getAndFeature()
pickle.dump(and_scores, open("C:\\Users\\sundaras\\AnacondaProjects\\OneML\\data\\and_features_dev.p", "wb"))

and2_scores = getAnd2Feature()
pickle.dump(and2_scores, open("C:\\Users\\sundaras\\AnacondaProjects\\OneML\\data\\and2_features_dev.p", "wb"))

In [ ]:
import pickle
bm25 = pickle.load(open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\bm25_features_dev.p", "rb" ))
len(bm25)

In [4]:
doclen = pickle.load(open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\doclen_features_dev.p", "rb" ))
len(doclen)

111397

In [5]:
coverage = pickle.load(open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\coverage_features_dev.p", "rb" ))
len(coverage)

111397

In [6]:
idf = pickle.load(open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\idf_features_dev.p", "rb" ))
len(idf)

111397

In [7]:
tf = pickle.load(open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\tf_features_dev.p", "rb" ))
len(tf)

111397

In [8]:
tfidf = pickle.load(open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\tfidf_features_dev.p", "rb" ))
len(tfidf)

111397

In [20]:
querylen = pickle.load(open( "C:\\Users\\mapyredd\\Documents\\marco\\data\\LTR\\querylen_features_dev.p", "rb" ))
len(querylen)

111397

In [8]:
querylen = pickle.load(open("C:\\Users\\sundaras\\AnacondaProjects\\OneML\\data\\and_features_dev.p", "rb"))
len(querylen)

111397

In [12]:
querylen = pickle.load(open("C:\\Users\\sundaras\\AnacondaProjects\\OneML\\data\\and2_features_dev.p", "rb"))
len(querylen)

111397